<hr>

# Chapter 01 — Why Reinforcement Learning Matters **Now**:
<hr>

**Theme**: RL as *decision-making under uncertainty* — when you can’t just label the right answer.

This course is notebook-first, where each chapter is represented as a notebook. Each notebook is designed to be read top-to-bottom and run as you go.

### What you will learn (3–5 bullets)
- What makes RL different from supervised learning (and from “just optimisation”).
- The core intuition: **actions influence future data**, so learning changes the world you observe.
- A first hands-on example: the **exploration vs exploitation** dilemma.

### Prereqs
- Python basics (functions, loops)
- Comfort reading simple plots

### Estimated time
- 45–60 minutes

<br>
<br>
<hr>
<br>
<br>

# 1. Introduction

Reinforcement learning matters now because we are increasingly building systems that don’t just *predict*, they **act**. They make decisions in the world, repeatedly, under uncertainty, with outcomes that arrive late and with feedback that is shaped by their own behavior. In settings like this, the core question isn’t *“what label matches this input?”* but **“what should I do next to achieve a long‑run goal?”** RL is the language and toolbox for that question.

A big driver is the rise of **autonomous systems**: software and machines that must operate in dynamic environments, where the “right” action depends on context and where mistakes can compound. An autonomous agent isn’t judged by a single output, but by the quality of an entire trajectory. How it gathers information, how it manages uncertainty, how it recovers from error, and how it balances competing objectives like speed, cost, reliability, and safety. Even when we can train components with supervised learning, the system-level behavior is fundamentally interactive. RL provides a way to directly optimise decision policies against outcomes that only make sense at the level of sequences.

**Robotics** makes this concrete. A robot arm learning to grasp, a quadruped learning to walk, or a drone learning to stabilise in wind all face the same reality, that actions change the future state of the world, sensors are noisy, and good control requires reasoning over time. You can’t realistically label the “correct” torque at every millisecond across every possible contact configuration. Instead, you specify outcomes—move efficiently, don’t fall, don’t collide, conserve energy, and learn behavior through interaction (often in simulation first). RL is compelling here not because it’s magic, but because it aligns with how control problems are structured: long horizons, delayed consequences, and the need for policies that adapt.

**Healthcare** is another domain where the RL framing is natural, and where it also forces you to confront what matters most, *constraints*. Clinical decisions are sequential. An intervention now can change patient state, influence which options are available later, and affect outcomes that only appear after hours or days. The objective is not next-step accuracy; it’s long-run patient outcomes under uncertainty, with strong safety requirements. RL provides a principled way to talk about sequential treatment policies, long-term return, and the tradeoffs between aggressive actions and risk, while also highlighting that naive reward maximisation is not enough. In many healthcare settings, the most important part of the RL problem is how to encode and enforce safety and how to learn from limited, biased, and ethically constrained data.

And then there’s the shift in AI itself: **LLMs and agents**. Modern language models increasingly operate as decision-makers, choosing when to ask a clarifying question, which tool to call, how to plan, and how to allocate computation. These are action choices with consequences, they change what information the system receives next, they incur cost, and they can fail in ways that only become apparent after a sequence of steps. This is exactly the terrain RL was built for: learning policies from interaction and optimising for outcomes that live at the level of trajectories (task success, user satisfaction, safety incidents), not just token-level likelihood.

Finally, RL has become central to **LLM training and fine-tuning**. Techniques like **RLHF** (reinforcement learning from human feedback) and related preference-optimisation approaches emerged because we care about properties such as helpfulness, harmlessness, honesty and instruction-following, which are hard to specify as a fixed supervised target for every prompt. Human preferences provide a *reward signal*, and RL provides a framework for turning that signal into improved behavior. Beyond RLHF, there’s increasing interest in more **end-to-end** RL-style training loops where models improve through interactions with environments, with tools, with users, and with automated evaluators. In these settings, the key RL questions reappear in modern form: how to explore safely, how to avoid reward hacking, how to evaluate policies robustly, how to ensure improvements generalise, and how to optimise what we truly want rather than what is easiest to measure.

So RL matters now not just because it’s a set of algorithms, but because it matches the shape of the problems we’re increasingly trying to solve. **Learning to act**, under uncertainty, over time, with *real* consequences.

<br>
<br>
<hr>
<br>
<br>

# 2. Core Idea

### RL in one sentence
**Reinforcement learning is learning to make good decisions from interaction.**

### The key twist
In supervised learning, the dataset is (mostly) fixed. In RL, **your actions determine what data you get next**.

That single fact creates most of RL’s “feel”:
- You must **explore** to discover better actions.
- Exploration can be costly (bad outcomes while you learn).
- Rewards are often noisy, delayed, and shaped by your choices.

### What RL is *not*
- **Not** just prediction: you can predict perfectly and still act poorly.
- **Not** just optimization: you optimize *a long-run objective* under feedback and uncertainty.
- **Not** always “deep”: classic tabular RL is foundational and still useful.

<br>
<br>
<hr>
<br>
<br>


# 3. Multi-armed bandit

We’ll start with the smallest possible interactive decision problem: a **multi-armed bandit**.

- You repeatedly choose one of several actions (arms).
- Each action gives a random reward with an unknown average.
- There is no “state” yet. That’s intentional, it isolates the core challenge.

Even here, RL-like thinking appears immediately:
- If you always pick the best arm *you currently believe in*, you may miss a better one.
- If you explore too much, you waste time on worse arms.


In [ ]:
# If you're running this in a fresh environment, you only need numpy + matplotlib.
# Uncomment below as required:

# %pip install numpy
# %pip install matplotlib

#=================================================================================
#=================================================================================

# Import libraries
import numpy as np
import matplotlib.pyplot as plt

# Generate a random seed for reproducability 
np.random.seed(7)

# Use matplotlib to generate parameters for plots
plt.rcParams["figure.figsize"] = (10, 4)
plt.rcParams["axes.grid"] = True


## Before we code: what “Gaussian bandit” means

In this bandit example, the list `means = [0.2, 0.5, 1.0, 0.7]` does **not** describe one “four-dimensional Gaussian.”
Instead, it defines **four separate 1D reward distributions** — one per action (arm):

- If you pick arm 0, reward $r \sim \mathcal{N}(0.2, \text{reward\_std}^2)$
- If you pick arm 1, reward $r \sim \mathcal{N}(0.5, \text{reward\_std}^2)$
- If you pick arm 2, reward $r \sim \mathcal{N}(1.0, \text{reward\_std}^2)$
- If you pick arm 3, reward $r \sim \mathcal{N}(0.7, \text{reward\_std}^2)$

So the bandit works like this: **you choose an action**, and then the environment samples a reward from *that arm’s* distribution.
There is **no state** and no transition dynamics in this toy problem; just repeated **action → reward**.

### Gotchas to keep in mind

- **Bandits are “stateless RL”**: this notebook focuses on the exploration–exploitation tradeoff without introducing states yet. (States come in the next notebooks.)
- **The reward is noisy**: even a worse arm can occasionally pay out more than the best arm on a single step. That’s why you need multiple samples and why early “greedy” choices can get stuck.
- **The best arm is fixed here**: this is a *stationary* bandit (means don’t change). Later we’ll discuss non-stationary settings where the best action can drift.
- **Reproducibility**: we set a random seed so your plots are repeatable. If you change the seed (or remove it), the exact curves will change—but the overall pattern should be similar.

### The code below initialises the Bandit for the later exercises

In [ ]:
class GaussianBandit:
    """
    K-armed bandit environment where each arm provides rewards drawn from a Gaussian (Normal) distribution.

    Each action (arm) i yields a reward sampled from N(mean_i, reward_std^2).
    Bandits are stateless, so the distribution parameters are fixed for each arm during an episode.
    """

    def __init__(self, means: np.ndarray, reward_std: float = 1.0):
        # Initialize the bandit environment.
        # Store the means as a numpy array to ensure correct type and operations
        self.means = np.array(means, dtype=float)
        # Store the standard deviation as float (all arms have the same std dev)
        self.reward_std = float(reward_std)

    @property
    def k(self) -> int:
        # Number of arms in the bandit.
        return int(self.means.shape[0])

    def step(self, action: int) -> float:
        #Sample a reward by pulling the specified arm.
        return float(np.random.normal(self.means[action], self.reward_std))

    def optimal_mean(self) -> float:
        # Return the true mean reward of the optimal arm.
        return float(np.max(self.means))

    def optimal_action(self) -> int:
        # Get the index of the optimal (highest-mean) action.
        return int(np.argmax(self.means))


# Example usage: create a bandit instance with 4 arms and check its parameters
bandit = GaussianBandit(means=np.array([0.2, 0.5, 1.0, 0.7]), reward_std=1.0)

# Print the true means and the index of the optimal arm
bandit.means, bandit.optimal_action()


<hr>

## 3.1 Exercise A

We’ll compare three simple strategies:
- **Pure exploitation**: always pick the arm with the highest estimated mean.
- **ε-greedy**: mostly exploit, but sometimes explore at random.
- **Pure exploration**: choose actions uniformly at random.

We’ll track:
- **Average reward** (how well we’re doing)
- **Regret** (how much reward we lost by not always taking the best arm)

Regret is a useful lens: it turns “learning efficiency” into a measurable quantity.


In [ ]:
def run_epsilon_greedy(
    bandit: GaussianBandit,
    n_steps: int,
    epsilon: float,
    initial_value: float = 0.0,
):
    """Run epsilon-greedy with incremental mean estimates."""
    k = bandit.k
    q = np.full(k, initial_value, dtype=float)  # estimated value of each action (Q-values)
    n = np.zeros(k, dtype=int)                  # number of times each action has been selected

    rewards = np.zeros(n_steps, dtype=float)    # rewards at each step
    actions = np.zeros(n_steps, dtype=int)      # actions taken at each step

    for t in range(n_steps):
        # With probability epsilon, choose a random arm (exploration)
        explore = np.random.rand() < epsilon
        if explore:
            a = np.random.randint(0, k)
        else:
            # Otherwise, exploit: pick the arm with current highest estimated value
            a = int(np.argmax(q))

        # Pull the selected arm and observe the reward
        r = bandit.step(a)

        n[a] += 1  # Update count of times arm a has been selected

        # Incremental mean update for Q-value of action a
        q[a] += (r - q[a]) / n[a]

        # Store the obtained reward and action
        rewards[t] = r
        actions[t] = a

    return rewards, actions


def summarise(bandit: GaussianBandit, rewards: np.ndarray):
    # Compute the optimal mean reward (for the best possible arm)
    optimal = bandit.optimal_mean()
    avg_reward = rewards.mean()
    regret = (optimal - rewards).cumsum()  # cumulative regret over time
    return avg_reward, regret

print('This cell has no output! (Except this...)')


## What this experiment is doing (plain English)

We’re going to simulate **$n\_steps$ interactions** with the bandit.

At each time step $t = 1, 2, \dots, n\_steps$:

1. The agent chooses an **action** (i.e., selects one arm to pull).
2. The bandit returns a **reward** by sampling from that arm’s reward distribution.
3. The agent updates its **estimate** of how good that arm is (based on the observed reward).

We’ll run the *same* bandit under **three different action-selection policies**:

- **Exploit-only ($\epsilon = 0.0$)**: always pull the arm with the highest current estimated value.
- **$\epsilon$-greedy ($\epsilon = 0.1$)**: usually exploit, but **10%** of the time pick a random arm to explore.
- **Random ($\epsilon = 1.0$)**: always pick a random arm (pure exploration).

Then we compare them using two metrics:

- **Average reward**: “How much reward did we get on average?”
- **Cumulative regret**: “How much reward did we miss out on compared to always pulling the truly best arm?”

> Important: even the best arm can give a low reward sometimes (because rewards are noisy), so early on it’s easy for a policy to get fooled. That’s why exploration can help.


### Run the below cell multiple times to observe differences in the resulting graphs!

In [ ]:
# Number of steps in the experiment
n_steps = 2000

# Run pure exploitation (epsilon=0.0: always greedy)
rewards_exploit, _ = run_epsilon_greedy(bandit, n_steps=n_steps, epsilon=0.0)

# Run epsilon-greedy: mix of exploration and exploitation
rewards_eps, _ = run_epsilon_greedy(bandit, n_steps=n_steps, epsilon=0.1)

# Run pure exploration (epsilon=1.0: always random)
rewards_random, _ = run_epsilon_greedy(bandit, n_steps=n_steps, epsilon=1.0)

# Compute average reward and cumulative regret for each strategy
avg_exploit, reg_exploit = summarise(bandit, rewards_exploit)
avg_eps, reg_eps = summarise(bandit, rewards_eps)
avg_random, reg_random = summarise(bandit, rewards_random)

# Print or display the average rewards for each strategy
avg_exploit, avg_eps, avg_random


def moving_average(x: np.ndarray, window: int = 50) -> np.ndarray:
    """Compute the moving average of array x with window size 'window'."""
    x = np.asarray(x)
    if window <= 1:
        # No smoothing if window is 1 or less
        return x
    kernel = np.ones(window) / window  # Uniform kernel for convolution
    return np.convolve(x, kernel, mode="valid")  # Apply moving average


import matplotlib.pyplot as plt

# Create a figure with 2 subplots side by side for reward and regret curves
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

# Left plot: Moving average reward
axs[0].plot(moving_average(rewards_exploit), label=f"exploit only (ε=0.0), avg={avg_exploit:.2f}")
axs[0].plot(moving_average(rewards_eps), label=f"ε-greedy (ε=0.1), avg={avg_eps:.2f}")
axs[0].plot(moving_average(rewards_random), label=f"random (ε=1.0), avg={avg_random:.2f}")
axs[0].axhline(bandit.optimal_mean(), color="black", linestyle="--", linewidth=1, label="optimal mean")
axs[0].set_title("Moving average reward (window=50)")
axs[0].set_xlabel("time step")
axs[0].set_ylabel("reward")
axs[0].legend()

# Right plot: Cumulative regret
axs[1].plot(reg_exploit, label="exploit only")
axs[1].plot(reg_eps, label="ε-greedy")
axs[1].plot(reg_random, label="random")
axs[1].set_title("Cumulative regret")
axs[1].set_xlabel("time step")
axs[1].set_ylabel("regret")
axs[1].legend()

plt.tight_layout()
plt.show()


### What you should notice
- **Exploit-only can get stuck** early if it commits to a suboptimal arm.
- **Random explores too much** and leaves reward on the table.
- **A little exploration** often wins: it pays a short-term cost to learn a better long-term behavior.

This is the “hello world” of RL: *acting to learn*.
<br>
<br>
<hr>
<br>
<br>



## 3.2 Exercise B: implement the update that makes the agent **learn**

So far, the agent is doing two things:
1. **Choosing actions** (which arm to pull)
2. **Updating its beliefs** about each arm’s value

In this exercise, you’ll implement the *belief update* step.

### Goal
Fill in the function `incremental_mean_update(...)` so that after the agent pulls arm $a$ and observes reward $r$, it updates its estimate $Q(a)$.

### The update rule
After you have pulled arm $a$ a total of $N(a)$ times, update:

$$
Q(a) \leftarrow Q(a) + \frac{1}{N(a)}(r - Q(a))
$$

Where:
- $Q(a)$ is the agent’s current estimate of the **average reward** for arm $a$
- $N(a)$ is **how many times you’ve pulled arm $a$ so far** (including this step)
- $r$ is the reward you just observed

### Intuition (why this makes sense)
- The term $(r - Q(a))$ is the **error** between what you observed and what you expected.
- The factor $\frac{1}{N(a)}$ makes updates **smaller over time** as you collect more data for that arm (because your estimate becomes more stable).

### Quick sanity check
Once implemented, `run_epsilon_greedy_v2(...)` should behave similarly to the earlier version of ε-greedy (average reward in the same ballpark, and it should learn to favor the best arm).

#### (Check your answer at the bottom of the notebook)

In [ ]:
def incremental_mean_update(q_a: float, n_a: int, r: float) -> float:
    """Return updated estimate for action a after observing reward r.

    q_a: current estimate
    n_a: *new* count after incrementing (so n_a >= 1)
    r: observed reward
    """
    # TODO: implement incremental mean update
    # Add your code here:
    # ........................................
    # ........................................
    # ........................................
    # ........................................
    return q_a


def run_epsilon_greedy_v2(
    bandit: GaussianBandit,
    n_steps: int,
    epsilon: float,
    initial_value: float = 0.0,
):
    k = bandit.k
    q = np.full(k, initial_value, dtype=float)
    n = np.zeros(k, dtype=int)

    rewards = np.zeros(n_steps, dtype=float)
    for t in range(n_steps):
        if np.random.rand() < epsilon:
            a = np.random.randint(0, k)
        else:
            a = int(np.argmax(q))

        r = bandit.step(a)
        n[a] += 1
        q[a] = incremental_mean_update(q[a], n[a], r)
        rewards[t] = r

    return rewards


# Sanity check: once you implement incremental_mean_update,
# this should behave similarly to the earlier implementation.
rewards_eps_v2 = run_epsilon_greedy_v2(bandit, n_steps=2000, epsilon=0.1)
rewards_eps_v2.mean()


<hr>

## Visual intuition: what learning looks like

Now that `incremental_mean_update` works, we can *see* learning more directly than by plotting noisy rewards.

We’ll plot two things:

1) **How often the agent chooses the optimal arm** (rolling window).
   - This shows whether the policy is actually improving.

2) The agent’s **value estimates** $Q(a)$ for each arm over time.
   - This shows what the update rule is doing internally.
   - We’ll also draw the **true means** as reference lines.

Because rewards are noisy, these plots are usually much easier to interpret than raw reward curves.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def rolling_mean(x: np.ndarray, window: int = 200) -> np.ndarray:
    x = np.asarray(x, dtype=float)
    if window <= 1:
        return x
    return np.convolve(x, np.ones(window) / window, mode="valid")

def trace_epsilon_greedy(
    bandit,
    n_steps: int,
    epsilon: float,
    initial_value: float = 0.0,
):
    """
    Runs epsilon-greedy while recording:
    - rewards[t]
    - actions[t]
    - q_history[t, a]  (the full Q vector over time)
    """
    k = bandit.k
    q = np.full(k, initial_value, dtype=float)
    n = np.zeros(k, dtype=int)

    rewards = np.zeros(n_steps, dtype=float)
    actions = np.zeros(n_steps, dtype=int)
    q_history = np.zeros((n_steps, k), dtype=float)

    for t in range(n_steps):
        if np.random.rand() < epsilon:
            a = np.random.randint(0, k)
        else:
            a = int(np.argmax(q))

        r = bandit.step(a)
        n[a] += 1
        q[a] = incremental_mean_update(q[a], n[a], r)

        rewards[t] = r
        actions[t] = a
        q_history[t] = q

    return rewards, actions, q_history

# --- Run experiments (fresh bandits so comparisons are fair) ---
n_steps = 4000
window = 200

np.random.seed(7)
bandit0 = GaussianBandit(means=np.array([0.2, 0.5, 1.0, 0.7]), reward_std=1.0)
opt_a0 = bandit0.optimal_action()
_, actions_exploit, _ = trace_epsilon_greedy(bandit0, n_steps, epsilon=0.0)

np.random.seed(7)
bandit1 = GaussianBandit(means=np.array([0.2, 0.5, 1.0, 0.7]), reward_std=1.0)
opt_a1 = bandit1.optimal_action()
_, actions_eps, q_hist_eps = trace_epsilon_greedy(bandit1, n_steps, epsilon=0.1)

np.random.seed(7)
bandit2 = GaussianBandit(means=np.array([0.2, 0.5, 1.0, 0.7]), reward_std=1.0)
opt_a2 = bandit2.optimal_action()
_, actions_random, _ = trace_epsilon_greedy(bandit2, n_steps, epsilon=1.0)

# --- Plot 1: rolling optimal-action rate ---
opt_rate_exploit = rolling_mean(actions_exploit == opt_a0, window=window)
opt_rate_eps = rolling_mean(actions_eps == opt_a1, window=window)
opt_rate_random = rolling_mean(actions_random == opt_a2, window=window)

plt.figure(figsize=(10, 4))
plt.plot(opt_rate_exploit, label="exploit-only (ε=0.0)")
plt.plot(opt_rate_eps, label="ε-greedy (ε=0.1)")
plt.plot(opt_rate_random, label="random (ε=1.0)")
plt.ylim(0, 1.05)
plt.title(f"How often we choose the optimal arm (rolling window={window})")
plt.xlabel("time step")
plt.ylabel("fraction optimal")
plt.legend()
plt.show()

# --- Plot 2: Q(a) estimates over time for ε-greedy ---
plt.figure(figsize=(10, 4))
for a in range(bandit1.k):
    plt.plot(rolling_mean(q_hist_eps[:, a], window=window), label=f"Q({a}) estimate")
    plt.axhline(bandit1.means[a], linestyle="--", linewidth=1, alpha=0.6)

plt.title(f"Value estimates Q(a) over time (ε=0.1, rolling window={window})")
plt.xlabel("time step")
plt.ylabel("Q estimate (smoothed)")
plt.legend()
plt.show()

<hr>

## 4. Diving Deeper

This section adds a bit more mathematical structure. You can skip it without affecting your ability to follow the rest of the notebook.

### Bandits, formally

A **$K$-armed bandit** is one of the simplest RL settings:

- There are $K$ actions (arms): $a \in \{1,\dots,K\}$.
- Each arm $a$ has an unknown reward distribution $P_a$ with unknown mean $\mu_a = \mathbb{E}[R \mid a]$.
- At each time step $t$, you choose an arm $A_t$ and observe a reward $R_t \sim P_{A_t}$.
- The environment is **stateless** and (in this notebook) **stationary**: the distributions $P_a$ do not change over time.

The learning problem is: choose actions to make the total reward large over time.

<br>
<br>
<hr style="border: none; border-top: 2px dashed #999; margin: 24px 0;">
<br>
<br>


### Expected reward and (pseudo-)regret

If you somehow knew the best arm in advance, you would always choose

$$
a^* = \arg\max_a \mu_a
\quad\text{with}\quad
\mu^* = \max_a \mu_a.
$$

To measure how costly learning is, we often use **regret**.

There are two closely related versions:

1) **Realised regret** (uses the random observed rewards):
$$
R_T = \sum_{t=1}^T (\mu^* - R_t)
$$
This can sometimes decrease on individual steps because rewards are noisy (you can get $R_t > \mu^*$ by luck).

2) **Pseudo-regret** (uses the true mean of the chosen arm):
$$
\bar{R}_T = \sum_{t=1}^T (\mu^* - \mu_{A_t})
$$
This is always $\ge 0$ and is often easier to reason about theoretically.

Both capture the same idea: **every time you don’t choose the best arm, you pay an opportunity cost.**

<br>
<br>
<hr style="border: none; border-top: 2px dashed #999; margin: 24px 0;">
<br>
<br>

### Why incremental updates show up everywhere

A basic idea in RL is to maintain an estimate $Q(a)$ of how good action $a$ is, and then improve it using new data.

In bandits, a natural target for $Q(a)$ is the action’s mean reward:
$$
Q(a) \approx \mu_a.
$$

If $Q_n$ is the average of the first $n$ rewards observed for a particular arm, then:
$$
Q_n = \frac{1}{n}\sum_{i=1}^n R_i
$$

From this definition you can derive the **online (incremental) mean update**:
$$
Q_n = Q_{n-1} + \frac{1}{n}(R_n - Q_{n-1}).
$$

This form is extremely important because it has the “RL update shape” you’ll see all course:
- **new estimate = old estimate + step-size × (target − old estimate)**

Here the target is the newest reward $R_n$, and the step-size is $\frac{1}{n}$.


<br>
<br>
<hr style="border: none; border-top: 2px dashed #999; margin: 24px 0;">
<br>
<br>



### Connecting back to exploration vs exploitation

Even if you use perfect averaging, you still face a decision problem:
- **Exploit**: choose the arm with the highest current estimate $Q(a)$
- **Explore**: try something uncertain to improve your estimates

ε-greedy is one of the simplest strategies for balancing this tradeoff.


### If you want to go deeper

- Sutton & Barto (2018), Chapter 2 (Multi-armed bandits)
- Lattimore & Szepesvári, *Bandit Algorithms* (more formal regret bounds and methods like UCB/Thompson Sampling)

<br>
<br>
<hr>

## 6. Knowledge Check

### Concept Checks:
1) In your own words, why does exploration change the *data distribution* you learn from?
2) Why can “exploit-only” be a bad strategy even when the world is stationary?
3) What does cumulative regret measure?

### Small coding change:
- Make the bandit harder by increasing `reward_std` to 2.0. What happens to regret? Why?

<br>
<br>
<hr>
<br>
<br>


## 7. Takeaways
- RL is about **sequential decision-making** with feedback.
- Actions influence the data you get, so learning and control are coupled.
- Even the simplest setting forces the exploration–exploitation tradeoff.

<br>
<br>
<hr>
<br>
<br>

## 8. What’s Next
- Next notebook: **RL loop + vocabulary** (state, action, reward, return, policy, value).
- Suggested reading: Sutton & Barto (2018), Chapter 2 (Bandits) for additional intuition.


In [ ]:
######################################################################
###################### ANSWERS BELOW #################################
######################################################################
###################### ANSWERS BELOW #################################
######################################################################
###################### ANSWERS BELOW #################################
######################################################################
            
                           #####
                           #####                                                 
                           #####                                                  
                           #####                                                    
                           #####                                                    
                        ###########                     
                         #########                       
                          #######                           
                           #####                             
                            ###                              
                             #                                
              

## Answer to Exersise B:

In [ ]:
def incremental_mean_update(q_a: float, n_a: int, r: float) -> float:
    """Return updated estimate for action a after observing reward r.

    q_a: current estimate
    n_a: *new* count after incrementing (so n_a >= 1)
    r: observed reward
    """
    # This is the added line of code that reflects the given update formula
    q_a = q_a + 1/n_a * (r - q_a) # <--------------------------------------
    return q_a

## Answers to Knowledge Check:
<hr style="border: none; border-top: 2px dashed #999; margin: 24px 0;">

### Concept checks: 

**1. Why does exploration change the data distribution you learn from?**

    Because in RL the data you collect depends on your actions. If you explore, you deliberately take actions you wouldn’t otherwise take, so you observe rewards/states from parts of the environment you’d miss under a purely greedy policy. Different policies ⇒ different action frequencies ⇒ different samples ⇒ different training data.

**2. Why can exploit-only be bad even when the world is stationary?**

    Early on your value estimates are noisy and based on few samples. If you exploit-only, you can “lock in” to an arm that looked best due to randomness and never gather enough evidence to discover a truly better arm. Stationary doesn’t mean “obvious from one sample”—it just means the underlying means don’t change.

**3. What does cumulative regret measure?**

    It measures the total opportunity cost of learning: how much reward you lost over time compared to an oracle that always picks the optimal arm. Lower cumulative regret means you learned faster / made better decisions sooner.

### Small coding change:

**- Increase reward_std to 2.0: what happens to regret, and why?**

    Regret typically increases (grows faster), because rewards become noisier. Noisier feedback makes it harder to tell which arm is truly best, so the agent needs more samples (and makes more mistakes) before it confidently exploits the optimal arm. You’ll often see slower improvement and more variability across runs.

